- This DL model implementation uses the Tensorflow's Functional API
- Depending on the cuDNN version and Tensorflow version, the  two Tensorflow-hack lines may need to be un-commented so that the notebook can see the GPU
- The output is very long, scroll down to the bottom to see the result as well as the corresponding model and parameters.
- The model is optimized for <b>precision</b>. We can re-run it for accuracy by changing the metrics and the parameter <b>monitor</b> in the ModelCheckpoint.

In [1]:
import tensorflow as tf

# Tensorflow hack to use GPU - 
#gpus= tf.config.experimental.list_physical_devices('GPU')
#tf.config.experimental.set_memory_growth(gpus[0], True)

# 
#import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras import Input, Model
from tensorflow.keras.layers import BatchNormalization, Flatten, Dense, Dropout
from tensorflow.keras.wrappers.scikit_learn import KerasClassifier
from tensorflow.keras.metrics import Precision
from tensorflow.keras.optimizers.schedules import ExponentialDecay
from tensorflow.keras.optimizers import SGD
import pandas as pd
import numpy as np
import datetime
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import Normalizer, MinMaxScaler
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import train_test_split
from sklearn.metrics import precision_score, accuracy_score
from tqdm.keras import TqdmCallback   # for cleaner training output
import random as python_random
from keras.callbacks import ModelCheckpoint   
import warnings
warnings.filterwarnings("ignore")



In [4]:
tf.__version__

'2.4.0'

In [ ]:
# for determination results
np.random.seed(0)
tf.random.set_seed(0)
python_random.seed(1)
random_state = 0 # this variable is used for sklearn classifiers

# load dataframe
train = pd.read_csv('weatherAUS.csv')
#train=train.sort_values(by='Date')
train=train.set_index('Date')

train = train.sample(frac=1).reset_index(drop=True)

#remove the entries without label
train = train[~train['RainTomorrow'].isna()]

#NA
train =train.dropna()

#convert Location, WindGustDir,WindDir9am to categorical features
le =LabelEncoder()
le.fit(train['Location'])
train['Location']=le.transform(train['Location'])
le.fit(train['WindGustDir'])
train['WindGustDir'] = le.transform(train['WindGustDir'])
le.fit(train['WindDir9am']) 
train['WindDir9am'] = le.transform(train['WindDir9am'])
le.fit(train['WindDir3pm']) 
train['WindDir3pm'] = le.transform(train['WindDir3pm'])

#convert the text labels to categorical
train['RainToday']=train['RainToday'].map(dict(Yes=1, No=0))
train['RainTomorrow']=train['RainTomorrow'].map(dict(Yes=1, No=0))
X = train.drop('RainTomorrow',axis=1)
y = train['RainTomorrow']

# train / test data sets
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state = random_state, test_size=0.1)
X_train, X_val, y_train, y_val = train_test_split(X, y, random_state = random_state)

#normalization
scaler = MinMaxScaler()
X_train = scaler.fit_transform(X_train)
X_val = scaler.transform(X_val)
X_test = scaler.transform(X_test)

# the deep learning model using TensorFlow's Functional API 
def keras_wrapper( optimizer):
    input = Input(shape=(21,), name="Rainfall Input")
    x = Dense(60, activation='relu', name="Dense-1")(input)
    x1 = Dense(40, activation='relu', name="Dense-2")(x)
    x = BatchNormalization(name="BN-1")(x1) # x1 here is intentioal, may be use for "shortcut" connections later                                            
    x = Dense(60, activation='relu', name="Dense-3")(x)
    x = Dropout(0.5, name="Dropout-1")(x)
    x = Dense(30, activation='relu',name="Dense-4")(x)
    x = BatchNormalization(name="BN-2")(x)
    x = Dense(20, activation='relu',name="Dense-5")(x)
    x = Dropout(0.5, name="Dropout-2")(x)
    x = Dense(40, activation='relu', name="Dense-6")(x)
    #x = x + x1
    x = BatchNormalization(name="BN-3")(x)
    x = Dense(40, activation='relu', name="Dense-7")(x)
    x = Dropout(0.5, name="Dropout-3")(x)
    output = Dense(1, activation='sigmoid', name="Output")(x)
    model = Model(inputs=input, outputs=output)
    model.compile( loss = "binary_crossentropy", optimizer="adam", metrics=['Precision']) 
    return model


In [3]:
precisions = []
optimizer = "adam"
for n in range(5):
    early_stopping_callback = tf.keras.callbacks.EarlyStopping(
    monitor="precision",
    min_delta=0,
    patience=60,
    verbose=1,
    mode="max",
    baseline=None,
    restore_best_weights=False,
    )
    lr_scheduler = ExponentialDecay(
    initial_learning_rate=1e-3,
    decay_steps=5000,
    decay_rate=0.9)

    lr_callback = tf.keras.callbacks.LearningRateScheduler(lr_scheduler)

    clf = keras_wrapper( optimizer = optimizer)
    checkpoint = ModelCheckpoint(save_best_only=True, verbose=1, filepath='model.weights.best.hdf5', monitor='precision', mode='max')
    history = clf.fit(X_train, y_train,batch_size=512, 
                  callbacks=[checkpoint,lr_callback, early_stopping_callback],
                  epochs=400,
                  validation_data=(X_val, y_val), verbose = 1 )

    clf.load_weights('model.weights.best.hdf5')
    y_pred = clf.predict(X_test)
    y_1 = y_pred > 0.5
    y_1 = y_1.astype(int)
    
    precisions.append(precision_score(y_test, y_1))
    print(f'\n***Precision {n}: {precisions[n]}')
print("\n==========")
print(f"Settings: early stopping {early_stopping_callback.monitor},{early_stopping_callback.patience}, learning_rate {lr_scheduler.get_config()}, model {clf}")
print(keras_wrapper(optimizer).summary())
print("\n==========")
print(f"Precisions: {precisions}")
print(f"Precision average: {np.mean(precisions)}")
print("\n==========")      
      

Epoch 1/400
83/83 [==============================] - 2s 8ms/step - loss: 0.7230 - precision: 0.2502 - val_loss: 0.5393 - val_precision: 0.0000e+00

Epoch 00001: precision improved from -inf to 0.29129, saving model to model.weights.best.hdf5
Epoch 2/400
83/83 [==============================] - 0s 4ms/step - loss: 0.4698 - precision: 0.5377 - val_loss: 0.4385 - val_precision: 1.0000

Epoch 00002: precision improved from 0.29129 to 0.57210, saving model to model.weights.best.hdf5
Epoch 3/400
83/83 [==============================] - 0s 5ms/step - loss: 0.4153 - precision: 0.6326 - val_loss: 0.4011 - val_precision: 1.0000

Epoch 00003: precision improved from 0.57210 to 0.63807, saving model to model.weights.best.hdf5
Epoch 4/400
83/83 [==============================] - 0s 4ms/step - loss: 0.3926 - precision: 0.6573 - val_loss: 0.3971 - val_precision: 0.9720

Epoch 00004: precision improved from 0.63807 to 0.66612, saving model to model.weights.best.hdf5
Epoch 5/400
83/83 [================


Epoch 00038: precision did not improve from 0.75638
Epoch 39/400
83/83 [==============================] - 0s 4ms/step - loss: 0.3325 - precision: 0.7410 - val_loss: 0.3427 - val_precision: 0.9009

Epoch 00039: precision did not improve from 0.75638
Epoch 40/400
83/83 [==============================] - 0s 5ms/step - loss: 0.3319 - precision: 0.7415 - val_loss: 0.3354 - val_precision: 0.8737

Epoch 00040: precision did not improve from 0.75638
Epoch 41/400
83/83 [==============================] - 0s 4ms/step - loss: 0.3244 - precision: 0.7468 - val_loss: 0.3514 - val_precision: 0.9144

Epoch 00041: precision did not improve from 0.75638
Epoch 42/400
83/83 [==============================] - 0s 4ms/step - loss: 0.3318 - precision: 0.7487 - val_loss: 0.3358 - val_precision: 0.8354

Epoch 00042: precision did not improve from 0.75638
Epoch 43/400
83/83 [==============================] - 0s 4ms/step - loss: 0.3229 - precision: 0.7497 - val_loss: 0.3522 - val_precision: 0.9178

Epoch 00043: p

83/83 [==============================] - 0s 4ms/step - loss: 0.3164 - precision: 0.7477 - val_loss: 0.3616 - val_precision: 0.9606

Epoch 00080: precision did not improve from 0.75638
Epoch 81/400
83/83 [==============================] - 0s 4ms/step - loss: 0.3204 - precision: 0.7564 - val_loss: 0.3645 - val_precision: 0.9667

Epoch 00081: precision did not improve from 0.75638
Epoch 82/400
83/83 [==============================] - 0s 4ms/step - loss: 0.3223 - precision: 0.7515 - val_loss: 0.3594 - val_precision: 0.9483

Epoch 00082: precision did not improve from 0.75638
Epoch 83/400
83/83 [==============================] - 0s 4ms/step - loss: 0.3173 - precision: 0.7495 - val_loss: 0.3608 - val_precision: 0.9518

Epoch 00083: precision did not improve from 0.75638
Epoch 84/400
83/83 [==============================] - 0s 4ms/step - loss: 0.3154 - precision: 0.7548 - val_loss: 0.3538 - val_precision: 0.9269

Epoch 00084: precision improved from 0.75638 to 0.75763, saving model to model.w

83/83 [==============================] - 0s 4ms/step - loss: 0.3072 - precision: 0.7637 - val_loss: 0.3684 - val_precision: 0.9596

Epoch 00121: precision improved from 0.75796 to 0.75856, saving model to model.weights.best.hdf5
Epoch 122/400
83/83 [==============================] - 0s 4ms/step - loss: 0.3129 - precision: 0.7465 - val_loss: 0.3696 - val_precision: 0.9569

Epoch 00122: precision did not improve from 0.75856
Epoch 123/400
83/83 [==============================] - 0s 4ms/step - loss: 0.3073 - precision: 0.7545 - val_loss: 0.3630 - val_precision: 0.9567

Epoch 00123: precision did not improve from 0.75856
Epoch 124/400
83/83 [==============================] - 0s 4ms/step - loss: 0.3145 - precision: 0.7451 - val_loss: 0.3705 - val_precision: 0.9610

Epoch 00124: precision did not improve from 0.75856
Epoch 125/400
83/83 [==============================] - 0s 4ms/step - loss: 0.3101 - precision: 0.7395 - val_loss: 0.3704 - val_precision: 0.9698

Epoch 00125: precision did not 


Epoch 00161: precision did not improve from 0.76237
Epoch 162/400
83/83 [==============================] - 0s 4ms/step - loss: 0.3092 - precision: 0.7591 - val_loss: 0.3769 - val_precision: 0.9738

Epoch 00162: precision improved from 0.76237 to 0.76297, saving model to model.weights.best.hdf5
Epoch 163/400
83/83 [==============================] - 0s 4ms/step - loss: 0.3043 - precision: 0.7660 - val_loss: 0.3568 - val_precision: 0.9484

Epoch 00163: precision improved from 0.76297 to 0.76493, saving model to model.weights.best.hdf5
Epoch 164/400
83/83 [==============================] - 0s 4ms/step - loss: 0.3034 - precision: 0.7646 - val_loss: 0.3627 - val_precision: 0.9561

Epoch 00164: precision did not improve from 0.76493
Epoch 165/400
83/83 [==============================] - 0s 4ms/step - loss: 0.3069 - precision: 0.7635 - val_loss: 0.3691 - val_precision: 0.9533

Epoch 00165: precision did not improve from 0.76493
Epoch 166/400
83/83 [==============================] - 0s 4ms/ste

...

83/83 [==============================] - 0s 5ms/step - loss: 0.2956 - precision: 0.7586 - val_loss: 0.3734 - val_precision: 0.9483

Epoch 00210: precision did not improve from 0.76578
Epoch 211/400
83/83 [==============================] - 0s 4ms/step - loss: 0.2968 - precision: 0.7566 - val_loss: 0.3778 - val_precision: 0.9738

Epoch 00211: precision did not improve from 0.76578
Epoch 212/400
83/83 [==============================] - 0s 5ms/step - loss: 0.2994 - precision: 0.7708 - val_loss: 0.3728 - val_precision: 0.9548

Epoch 00212: precision improved from 0.76578 to 0.76794, saving model to model.weights.best.hdf5
Epoch 213/400
83/83 [==============================] - 1s 6ms/step - loss: 0.2949 - precision: 0.7671 - val_loss: 0.3813 - val_precision: 0.9521

Epoch 00213: precision did not improve from 0.76794
Epoch 214/400
83/83 [==============================] - 0s 5ms/step - loss: 0.2968 - precision: 0.7568 - val_loss: 0.4023 - val_precision: 0.9701

Epoch 00214: precision did not 

83/83 [==============================] - 0s 5ms/step - loss: 0.2933 - precision: 0.7612 - val_loss: 0.4225 - val_precision: 0.9882

Epoch 00251: precision did not improve from 0.76794
Epoch 252/400
83/83 [==============================] - 0s 5ms/step - loss: 0.2929 - precision: 0.7634 - val_loss: 0.4457 - val_precision: 0.9950

Epoch 00252: precision did not improve from 0.76794
Epoch 253/400
83/83 [==============================] - 0s 5ms/step - loss: 0.2974 - precision: 0.7604 - val_loss: 0.4105 - val_precision: 0.9691

Epoch 00253: precision did not improve from 0.76794
Epoch 254/400
83/83 [==============================] - 0s 5ms/step - loss: 0.2948 - precision: 0.7687 - val_loss: 0.3841 - val_precision: 0.9592

Epoch 00254: precision did not improve from 0.76794
Epoch 255/400
83/83 [==============================] - 0s 5ms/step - loss: 0.2966 - precision: 0.7593 - val_loss: 0.3998 - val_precision: 0.9500

Epoch 00255: precision did not improve from 0.76794
Epoch 256/400
83/83 [===

83/83 [==============================] - 0s 5ms/step - loss: 0.2932 - precision: 0.7764 - val_loss: 0.3803 - val_precision: 0.9603

Epoch 00292: precision did not improve from 0.77106
Epoch 293/400
83/83 [==============================] - 0s 5ms/step - loss: 0.2944 - precision: 0.7614 - val_loss: 0.4048 - val_precision: 0.9817

Epoch 00293: precision did not improve from 0.77106
Epoch 294/400
83/83 [==============================] - 0s 5ms/step - loss: 0.2896 - precision: 0.7739 - val_loss: 0.4103 - val_precision: 0.9875

Epoch 00294: precision did not improve from 0.77106
Epoch 295/400
83/83 [==============================] - 0s 5ms/step - loss: 0.2904 - precision: 0.7717 - val_loss: 0.3894 - val_precision: 0.9690

Epoch 00295: precision did not improve from 0.77106
Epoch 296/400
83/83 [==============================] - 0s 5ms/step - loss: 0.2890 - precision: 0.7616 - val_loss: 0.3879 - val_precision: 0.9609

Epoch 00296: precision did not improve from 0.77106
Epoch 297/400
83/83 [===

83/83 [==============================] - 0s 5ms/step - loss: 0.2877 - precision: 0.7770 - val_loss: 0.3890 - val_precision: 0.9758

Epoch 00333: precision did not improve from 0.77443
Epoch 334/400
83/83 [==============================] - 0s 5ms/step - loss: 0.2884 - precision: 0.7619 - val_loss: 0.3973 - val_precision: 0.9627

Epoch 00334: precision did not improve from 0.77443
Epoch 335/400
83/83 [==============================] - 0s 5ms/step - loss: 0.2818 - precision: 0.7655 - val_loss: 0.4174 - val_precision: 0.9864

Epoch 00335: precision did not improve from 0.77443
Epoch 336/400
83/83 [==============================] - 0s 5ms/step - loss: 0.2922 - precision: 0.7689 - val_loss: 0.4181 - val_precision: 0.9769

Epoch 00336: precision did not improve from 0.77443
Epoch 337/400
83/83 [==============================] - 0s 5ms/step - loss: 0.2843 - precision: 0.7713 - val_loss: 0.3915 - val_precision: 0.9570

Epoch 00337: precision did not improve from 0.77443
Epoch 338/400
83/83 [===

83/83 [==============================] - 0s 5ms/step - loss: 0.2811 - precision: 0.7635 - val_loss: 0.3884 - val_precision: 0.9622

Epoch 00374: precision did not improve from 0.77443
Epoch 375/400
83/83 [==============================] - 0s 5ms/step - loss: 0.2869 - precision: 0.7683 - val_loss: 0.3942 - val_precision: 0.9620

Epoch 00375: precision did not improve from 0.77443
Epoch 376/400
83/83 [==============================] - 0s 5ms/step - loss: 0.2873 - precision: 0.7693 - val_loss: 0.3890 - val_precision: 0.9584

Epoch 00376: precision did not improve from 0.77443
Epoch 377/400
83/83 [==============================] - 0s 5ms/step - loss: 0.2921 - precision: 0.7764 - val_loss: 0.4091 - val_precision: 0.9847

Epoch 00377: precision did not improve from 0.77443
Epoch 378/400
83/83 [==============================] - 0s 5ms/step - loss: 0.2862 - precision: 0.7656 - val_loss: 0.4172 - val_precision: 0.9841

Epoch 00378: precision did not improve from 0.77443
Epoch 379/400
83/83 [===